In [40]:
%load_ext autoreload
%autoreload 2

# standard libraries
import sys

sys.path.append("../")

import os
import tiktoken
import time
import torch
from typing import List, Tuple
from math import ceil

# external libraries
import pandas as pd
import numpy as np
from llama_index.text_splitter import SentenceSplitter  # one of the best on the market
from rich import print
from rich.pretty import pprint  # nifty library for pretty printing
from sentence_transformers import SentenceTransformer, losses, InputExample, models
from torch import cuda
from tqdm import tqdm

# external files
try:
    from preprocessing import FileIO
except ModuleNotFoundError:
    from src.preprocessor.preprocessing import FileIO

from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(), override=True)

from src.database.weaviate_interface_v4 import WeaviateIndexer, WeaviateWCS
from src.database.database_utils import get_weaviate_client

import os
import time
import json
from typing import List
from tqdm import tqdm
from rich import (
    print,
)  # nice library that provides improved printing output (overrides default print function)

from src.database.properties_template import properties

api_key = os.environ["WEAVIATE_API_KEY"]
url = os.environ["WEAVIATE_ENDPOINT"]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


sys:1: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x7f3dcda0cb20>


In [41]:
def split_contents(
    corpus: list[dict], text_splitter: SentenceSplitter, content_field: str = "content"
) -> list[list[str]]:
    """
    Given a corpus of "documents" with text content, this function splits the
    content field into chunks sizes as specified by the text_splitter.

    Example
    -------
    corpus = [
            {'title': 'This is a cool show', 'content': 'There is so much good content on this show. \
              This would normally be a really long block of content. ... But for this example it will not be.'},
            {'title': 'Another Great Show', 'content': 'The content here is really good as well.  If you are \
              reading this you have too much time on your hands. ... More content, blah, blah.'}
           ]

    output = split_contents(data, text_splitter, content_field="content")

    output >>> [['There is so much good content on this show.', 'This would normally be a really long block of content.', \
                 'But for this example it will not be'],
                ['The content here is really good as well.', 'If you are reading this you have too much time on your hands.', \
                 'More content, blah, blah.']
                ]
    """

    ########################
    # START YOUR CODE HERE #
    ########################
    output = []
    for doc in tqdm(corpus):
        output.append(text_splitter.split_text(doc[content_field]))

    return output


def encode_content_splits(
    content_splits: list[list[str]],
    model: SentenceTransformer,
    device: str = "cuda:0" if cuda.is_available() else "cpu",
) -> list[list[tuple[str, list[float]]]]:
    """
    Encode content splits as vector embeddings from a vectors of content splits
    where each vectors of splits is a single podcast episode.

    Example
    -------
    content_splits =  [['There is so much good content on this show.', 'This would normally be a really long block of content.'],
                       ['The content here is really good as well.', 'More content, blah, blah.']
                      ]

    output = encode_content_splits(content_splits, model)

    output >>> [
          EPISODE 1 -> [('There is so much good content on this show.',[ 1.78036056e-02, -1.93265956e-02,  3.61164124e-03, -5.89650944e-02,
                                                                         1.91510320e-02,  1.60808843e-02,  1.13610983e-01,  3.59948091e-02,
                                                                        -1.73066761e-02, -3.30348089e-02, -1.00898169e-01,  2.34847311e-02]
                                                                        )
                         tuple(text, vectors), tuple(text, vectors), tuple(text, vectors)....],
          EPISODE 2 ->  [tuple(text, vectors), tuple(text, vectors), tuple(text, vectors)....],
          EPISODE n ... [tuple(text, vectors), tuple(text, vectors), tuple(text, vectors)....]
    """

    text_vector_tuples = []

    ########################
    # START YOUR CODE HERE #
    ########################

    model.to(device)

    for content in tqdm(content_splits):
        vecs = model.encode(content).tolist()
        text_vector = [(t, v) for t, v in zip(content, vecs)]
        text_vector_tuples.append(text_vector)

    return text_vector_tuples


def join_metadata(
    corpus: list[dict],
    text_vector_list: list[list[tuple[str, list]]],
    unique_id_field: str = "video_id",
    content_field: str = "content",
    embedding_field: str = "content_embedding",
) -> list[dict]:
    """
    Combine episode metadata from original corpus with text/vectors tuples.
    Creates a new dictionary for each text/vector combination.
    """

    joined_documents = []

    ########################
    # START YOUR CODE HERE #
    ########################

    for i, doc in enumerate(corpus):
        for j, tv in enumerate(text_vector_list[i]):
            corp_dict = {key: value for key, value in doc.items() if key != "content"}
            video_id = doc["video_id"]
            corp_dict["doc_id"] = f"{video_id}_{j}"
            corp_dict["content"] = tv[0]
            corp_dict["content_embedding"] = tv[1]
            joined_documents.append(corp_dict)

    return joined_documents


def create_dataset(
    corpus: list[dict],
    embedding_model: SentenceTransformer,
    text_splitter: SentenceSplitter,
    save_to_disk: bool,
    file_outpath: str = None,
    unique_id_field: str = "video_id",
    content_field: str = "content",
    embedding_field: str = "content_embedding",
    device: str = "cuda:0" if cuda.is_available() else "cpu",
) -> list[dict]:
    """
    Given a raw corpus of data, this function creates a new dataset where each dataset
    doc contains episode metadata and it's associated text chunk and vector representation.
    Output is directly saved to disk.
    """
    if save_to_disk and not file_outpath:
        raise ValueError(
            f"Saving to disk is enabled but file_outpath was left as a None value.\n\
            Enter a valid file_outpath or mark save_to_disk as False"
        )

    io = FileIO()

    chunk_size = text_splitter.chunk_size
    print(f"Creating dataset using chunk_size: {chunk_size}")
    start = time.perf_counter()
    ########################
    # START YOUR CODE HERE #
    ########################
    content_splits = split_contents(corpus, text_splitter)
    text_vector_tuples = encode_content_splits(content_splits, embedding_model)
    joined_docs = join_metadata(corpus, text_vector_tuples)
    ########################
    # END YOUR CODE HERE #
    ########################
    if save_to_disk:
        io.save_as_parquet(file_path=file_outpath, data=joined_docs, overwrite=False)
    end = time.perf_counter() - start
    print(
        f"Total Time to process dataset of chunk_size ({chunk_size}): {round(end/60, 2)} minutes"
    )
    return joined_docs

In [42]:
# root folder on Google Colab is: /content/
root_folder = "../data/"
data_file = "huberman_labs.json"
data_path = os.path.join(root_folder, data_file)
data_path

data = FileIO.load_json(data_path)

In [7]:
def load_pretrained_model(model_name: str = "sentence-transformers/all-MiniLM-L6-v2"):
    """
    Loads sentence transformer modules and returns a pretrained
    model for finetuning.
    """
    word_embedding_model = models.Transformer(model_name_or_path=model_name)
    pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
    model = SentenceTransformer(modules=[word_embedding_model, pooling_model])
    model.to("cuda")
    return model

In [31]:
# define the model you want to use
model_names = [
    "../models/bge-base-finetuned-500",
]

base = ["bge_finetuned_500"]

In [14]:
chunk_sizes = [256, 512]
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo-0125")

for chunk_size in chunk_sizes:
    for model_name, bas in zip(model_names, base):
        gpt35_txt_splitter = SentenceSplitter(
            chunk_size=chunk_size, tokenizer=encoding.encode, chunk_overlap=0
        )
        outpath = f"../data/huberman_{bas}_{chunk_size}"
        model = load_pretrained_model(model_name)
        create_dataset(
            data, model, gpt35_txt_splitter, save_to_disk=True, file_outpath=outpath
        )

        client = WeaviateWCS(
            endpoint=url, api_key=api_key, model_name_or_path=model_name
        )

        data_path = f"../data/huberman_{bas}_{chunk_size}.parquet"

        data = FileIO.load_parquet(data_path)

        collection_name = f"Huberman_{bas}_{chunk_size}"

        client.create_collection(
            collection_name=collection_name,
            properties=properties,
            description="Huberman Labs: 193 full-length transcripts",
        )

        indexer = WeaviateIndexer(client)

        batch_object = indexer.batch_index_data(data, collection_name)

Creating dataset using chunk_size: 128

100%|██████████| 193/193 [06:38<00:00,  2.06s/it]


FileExistsError: File by name huberman_bge_finetuned_500_128.parquet already exists, try using another file name or set overwrite to True.

In [35]:
chunk_size = 128
# define the model you want to use
model_names = [
    "../models/bge-base-finetuned-500",
]

base = ["bge_finetuned_500"]

model_name = model_names[0]
bas = base[0]

client = WeaviateWCS(endpoint=url, api_key=api_key, model_name_or_path=model_name)
try:
    data_path = f"../data/huberman_{bas}_{chunk_size}.parquet"

    data = FileIO.load_parquet(data_path)

    collection_name = f"Huberman_{bas}_{chunk_size}"

    client.create_collection(
        collection_name=collection_name,
        properties=properties,
        description="Huberman Labs: 193 full-length transcripts",
    )

    indexer = WeaviateIndexer(client)

    batch_object = indexer.batch_index_data(data, collection_name)
except:
    print("rip")

client.close()

Shape of data: (51128, 13)
Memory Usage: 5.07+ MB
Collection "Huberman_bge_finetuned_500_128" created


 11%|█         | 5605/51128 [00:05<00:27, 1626.26it/s]/opt/conda/envs/vec/lib/python3.10/site-packages/pydantic/_internal/_model_construction.py:253: ResourceWarning: unclosed <ssl.SSLSocket fd=106, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('10.162.0.2', 53866), raddr=('34.149.137.116', 443)>
  return hasattr(instance, '__pydantic_validator__') and super().__instancecheck__(instance)
 79%|███████▉  | 40592/51128 [02:20<00:16, 639.44it/s] /opt/conda/envs/vec/lib/python3.10/site-packages/weaviate/warnings.py:219: UserWarning: Bat003: The dynamic batch-size could not be refreshed successfully with error UnexpectedStatusCodeError('Nodes status! Unexpected status code: 502, with response body: None.')
  warnings.warn(
 81%|████████  | 41503/51128 [02:22<00:33, 290.60it/s]

Upload errors exceed error_threshold...
Processing finished in 2.38 minutes.
Batch job failed. Review errors using these keys: ['num_errors', 'error_messages', 'doc_ids']



/opt/conda/envs/vec/lib/python3.10/site-packages/weaviate/warnings.py:219: UserWarning: Bat003: The dynamic batch-size could not be refreshed successfully with error RemoteProtocolError('Server disconnected without sending a response.')
  warnings.warn(


sys:1: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x7f3dcdce6260>


In [43]:
data_path = f"../data/huberman_{bas}_{chunk_size}.parquet"

data = FileIO.load_parquet(data_path)

Shape of data: (51128, 13)
Memory Usage: 5.07+ MB


In [44]:
for d in data:
    if d["doc_id"] == "LVxL_p_kToc_201":
        print(d)

{
    'video_id': 'LVxL_p_kToc',
    'title': 'Dr. Erich Jarvis: The Neuroscience of Speech, Language & Music | Huberman Lab Podcast #87',
    'length_seconds': 6860,
    'keywords': [
        'andrew huberman',
        'huberman lab podcast',
        'huberman podcast',
        'dr. andrew huberman',
        'neuroscience',
        'huberman lab',
        'andrew huberman podcast',
        'the huberman lab podcast',
        'science podcast',
        'Erich Jarvis',
        'science of speech',
        'science of language',
        'science of music',
        'science of movement',
        'Rockefeller University professor'
    ],
    'view_count': 189852,
    'episode_num': 88,
    'episode_url': 'https://www.youtube.com/watch?v=LVxL_p_kToc',
    'thumbnail_url': 'https://i.ytimg.com/vi/LVxL_p_kToc/mqdefault.jpg',
    'guest': 'Dr. Erich Jarvis',
    'summary': "In episode 87 of the Huberman Lab podcast, Dr. Andrew Huberman interviews Dr. Erich Jarvis, a 
neurobiologist exploring the complexities of vocal learning, speech, and language. Dr. Jarvis's work investigates 
the genetic and neural underpinnings of how species like humans, songbirds, and parrots acquire and produce 
language, including the ability to learn and articulate complex vocalizations. They discuss the intricate 
relationship between speech, language, and movement, such as dance, and how these elements are intertwined both 
biologically and cognitively. They also delve into the idea that reading engages sub-perceptual motor activity in 
the throat, reflecting internalized speech. The conversation covers the potential evolutionary pathway from innate 
vocalizations to learned speech and the concept of critical periods for language acquisition. Dr. Jarvis's research
suggests that the brain regions associated with speech production may have evolved from those involved in 
controlling body movement, leading to the unique human capacity for sophisticated language and even the ability to 
dance",
    'doc_id': 'LVxL_p_kToc_201',
    'content': "Yeah, we have some things cooking. And thank you for inviting me here to get the word out to the 
community of what's going on in the science world. Well, we're honored and very grateful to you, Eric. Thank you. 
You're welcome. Thank you for joining me today for my discussion with Dr. Eric Jarvis. If you'd like to learn more 
about his laboratory's work, you can go to Jarvis Lab, spelled J-A-R-V-I-S, lab, all one word, jarvislab.net.",
    'content_embedding': [
        0.4429281949996948,
        0.001935548149049282,
        -0.7133868336677551,
        0.13902318477630615,
        0.9004620909690857,
        -0.17185832560062408,
        -0.12739121913909912,
        0.5825206637382507,
        -0.3299395442008972,
        -0.7870659232139587,
        -0.47945886850357056,
        -0.3896726667881012,
        -0.19670535624027252,
        0.5282865166664124,
        -0.13482657074928284,
        0.6195561289787292,
        0.3340061604976654,
        0.7410432696342468,
        -0.49694448709487915,
        1.1736942529678345,
        0.5987666845321655,
        0.1619165539741516,
        0.2590060532093048,
        0.14149661362171173,
        -0.04795587807893753,
        0.3169953227043152,
        0.02942252904176712,
        0.18470001220703125,
        -0.7669554352760315,
        -1.1378397941589355,
        0.7827674150466919,
        0.867953896522522,
        1.2564053535461426,
        -0.7565804719924927,
        -0.25562962889671326,
        -0.044499412178993225,
        -0.21978966891765594,
        0.12206325680017471,
        -0.5456921458244324,
        0.7103585004806519,
        -0.05775223672389984,
        0.4736984670162201,
        -0.10643866658210754,
        0.31078454852104187,
        -0.37018680572509766,
        -0.123226098716259,
        0.47295957803726196,
        -0.07055200636386871,
        -0.8029377460479736,
        0.2685026228427887,
        0.05976393073797226,
        0.23